In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
import os

# === Path to spectrogram images ===
data_dir = "Train_Spectrogram_Images"

# === Image Preprocessing ===
img_size = (300, 300)  # EfficientNetB3 requires at least 300x300 input
batch_size = 32

# Data augmentation and rescaling
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# === Load Pretrained EfficientNetB3 base model ===
base_model = EfficientNetB3(include_top=False,
                            weights='imagenet',
                            input_shape=(img_size[0], img_size[1], 3))

# === Freeze base model ===
base_model.trainable = False  # You can fine-tune later

# === Add custom classification head ===
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(train_generator.num_classes, activation='softmax')(x)

# === Final model ===
model = Model(inputs=base_model.input, outputs=output_layer)

# === Compile model ===
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# === Train model ===
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator